In [1]:
import numpy as np

In [2]:
# 載入資料
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

In [3]:
# 重塑數據形狀為 (樣本數, 高度, 寬度, 頻道數)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(2486, 64, 690, 1)
(311, 64, 690, 1)
(311, 64, 690, 1)


In [4]:
print(X_train[15])

[[[0.75954854]
  [0.9417032 ]
  [0.97626275]
  ...
  [0.89563864]
  [0.87509805]
  [0.8517748 ]]

 [[0.7634915 ]
  [0.89372426]
  [0.9136159 ]
  ...
  [0.8338712 ]
  [0.7947523 ]
  [0.79095936]]

 [[0.7207955 ]
  [0.7435139 ]
  [0.75551146]
  ...
  [0.7425578 ]
  [0.7344607 ]
  [0.6680453 ]]

 ...

 [[0.2105926 ]
  [0.23406295]
  [0.22235227]
  ...
  [0.19235858]
  [0.1849844 ]
  [0.20453115]]

 [[0.19425431]
  [0.20613436]
  [0.17414455]
  ...
  [0.16726056]
  [0.18581752]
  [0.21731324]]

 [[0.14248542]
  [0.14009257]
  [0.09383097]
  ...
  [0.06196117]
  [0.12550421]
  [0.18973407]]]


In [29]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.keras.layers import BatchNormalization

def create_model(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=output)
    return model


In [30]:
input_shape = np.shape(X_train)[1:]
model = create_model(input_shape)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64, 690, 1)]      0         
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 690, 32)       320       
                                                                 
 batch_normalization_9 (Batc  (None, 64, 690, 32)      128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 32, 345, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 345, 64)       18496     
                                                                 
 batch_normalization_10 (Bat  (None, 32, 345, 64)      256 

# 編譯模型

In [31]:
from keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[reduce_lr], verbose=2)


Epoch 1/50
78/78 - 4s - loss: 2.3545 - accuracy: 0.5205 - val_loss: 0.6920 - val_accuracy: 0.5338 - lr: 0.0010 - 4s/epoch - 51ms/step
Epoch 2/50
78/78 - 3s - loss: 1.1197 - accuracy: 0.5302 - val_loss: 0.6935 - val_accuracy: 0.5338 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 3/50
78/78 - 3s - loss: 0.7989 - accuracy: 0.5048 - val_loss: 0.6907 - val_accuracy: 0.5338 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 4/50
78/78 - 3s - loss: 0.7281 - accuracy: 0.5350 - val_loss: 0.7055 - val_accuracy: 0.4630 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 5/50
78/78 - 3s - loss: 0.7026 - accuracy: 0.5237 - val_loss: 0.6984 - val_accuracy: 0.4759 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 6/50
78/78 - 3s - loss: 0.6882 - accuracy: 0.5579 - val_loss: 0.6829 - val_accuracy: 0.5916 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 7/50
78/78 - 3s - loss: 0.6788 - accuracy: 0.5664 - val_loss: 0.6715 - val_accuracy: 0.5498 - lr: 0.0010 - 3s/epoch - 33ms/step
Epoch 8/50
78/78 - 3s - loss: 0.6660 - accuracy: 0.5813 - val_

# 訓練模型

In [23]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16, verbose=2)

Epoch 1/50
156/156 - 2s - loss: 0.6933 - accuracy: 0.5097 - val_loss: 0.6875 - val_accuracy: 0.6013 - 2s/epoch - 12ms/step
Epoch 2/50
156/156 - 1s - loss: 0.6758 - accuracy: 0.5788 - val_loss: 0.6823 - val_accuracy: 0.5820 - 1s/epoch - 9ms/step
Epoch 3/50
156/156 - 1s - loss: 0.6528 - accuracy: 0.6207 - val_loss: 0.6290 - val_accuracy: 0.6817 - 1s/epoch - 10ms/step
Epoch 4/50
156/156 - 1s - loss: 0.6205 - accuracy: 0.6585 - val_loss: 0.6014 - val_accuracy: 0.7010 - 1s/epoch - 10ms/step
Epoch 5/50
156/156 - 2s - loss: 0.6055 - accuracy: 0.6806 - val_loss: 0.5926 - val_accuracy: 0.6785 - 2s/epoch - 10ms/step
Epoch 6/50
156/156 - 1s - loss: 0.5794 - accuracy: 0.7047 - val_loss: 0.6027 - val_accuracy: 0.6913 - 1s/epoch - 10ms/step
Epoch 7/50
156/156 - 2s - loss: 0.5773 - accuracy: 0.7092 - val_loss: 0.5737 - val_accuracy: 0.6945 - 2s/epoch - 10ms/step
Epoch 8/50
156/156 - 2s - loss: 0.5705 - accuracy: 0.7261 - val_loss: 0.5772 - val_accuracy: 0.6945 - 2s/epoch - 10ms/step
Epoch 9/50
156/15

KeyboardInterrupt: 